In [1]:
# 导入库与基础配置
import os
from pathlib import Path
import random
import json
from datetime import datetime

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification

# 设备与随机种子
device = torch.device("mps" if torch.backends.mps.is_available() else ("cuda" if torch.cuda.is_available() else "cpu"))
print("Using device:", device)

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# checkpoint 目录
CHECKPOINT_DIR = Path("model/checkpoint_model1")
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)
LAST_CKPT = CHECKPOINT_DIR / "model1_last.pt"
BEST_CKPT = CHECKPOINT_DIR / "model1_best.pt"

print("Checkpoint dir:", CHECKPOINT_DIR.resolve())


Using device: mps
Checkpoint dir: /Users/jylu/Projects/acafeed/model/checkpoint_model1


In [10]:
# 定义数据集与多标签模型（Model1）

import pandas as pd
from datasets import Dataset

# 为了复用原始数据，这里从现有的 arxiv JSON 构建与 model.ipynb 一致的数据流程
file_path = os.path.expanduser("~/.cache/kagglehub/datasets/Cornell-University/arxiv/versions/259/arxiv-metadata-oai-snapshot.json")

chunks = pd.read_json(file_path, lines=True, chunksize=100000)
dfs = []
for chunk in chunks:
    dfs.append(chunk[["title", "categories"]])

df_small = pd.concat(dfs, ignore_index=True)
raw_df = df_small.rename(columns={"categories": "Category", "title": "Title"})
raw_df["Category"] = raw_df["Category"].str.split(" ")
raw_df["Title"] = raw_df["Title"].str.strip()

# 统计类别并过滤低频类别
all_categories = [c for sub in raw_df["Category"] for c in sub]
category_counts = pd.Series(all_categories).value_counts()
filtered_categories = category_counts[category_counts >= 20].index.tolist()

category_to_id = {cat: idx for idx, cat in enumerate(filtered_categories)}
label2id = category_to_id
id2label = {v: k for k, v in category_to_id.items()}

import numpy as np

def encode_labels(cats):
    labels = np.zeros(len(category_to_id), dtype=float)
    for cat in cats:
        if cat in category_to_id:
            labels[category_to_id[cat]] = 1.0
    return labels

filtered_df = raw_df[raw_df["Category"].apply(lambda cats: any(cat in filtered_categories for cat in cats))].reset_index(drop=True)
filtered_df["label"] = filtered_df["Category"].apply(encode_labels)

from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(filtered_df, test_size=0.05, random_state=SEED)
print(f"Training samples: {len(train_df)}, Testing samples: {len(test_df)}")
print(f"Number of labels: {len(category_to_id)}")

# 转为 HF Dataset
# 限制个数
TRAINNUM = 50000
TESTNUM = 100
# Randomly select a subset for quicker testing
train_ds = Dataset.from_pandas(train_df.reset_index(drop=True).sample(TRAINNUM, random_state=SEED)).shuffle(seed=SEED)
test_ds = Dataset.from_pandas(test_df.reset_index(drop=True).sample(TESTNUM, random_state=SEED)).shuffle(seed=SEED)

max_length = 64
tokenizer_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)


def tokenize(batch):
    tokenized = tokenizer(batch["Title"], truncation=True, padding="max_length", max_length=max_length)
    tokenized["labels"] = batch["label"]
    return tokenized

train_tokenized = train_ds.map(tokenize, batched=True)
test_tokenized = test_ds.map(tokenize, batched=True)

cols = ["input_ids", "attention_mask", "labels"]
train_tokenized = train_tokenized.remove_columns([c for c in train_tokenized.column_names if c not in cols])
test_tokenized = test_tokenized.remove_columns([c for c in test_tokenized.column_names if c not in cols])

train_tokenized.set_format(type="torch")
test_tokenized.set_format(type="torch")

print(f"训练集: {len(train_tokenized)} 样本, 测试集: {len(test_tokenized)} 样本")

# 定义模型（多标签 DistilBERT）
from transformers import AutoConfig, AutoModelForSequenceClassification

base_model_name = "distilbert/distilbert-base-uncased"
config = AutoConfig.from_pretrained(base_model_name)
config.num_labels = len(category_to_id)
config.problem_type = "multi_label_classification"
config.id2label = id2label
config.label2id = label2id

model = AutoModelForSequenceClassification.from_pretrained(
    base_model_name,
    config=config,
    ignore_mismatched_sizes=True,
)
model.to(device)

print("Model initialized:", model.__class__.__name__)
print("Num labels:", model.config.num_labels)
print("Problem type:", model.config.problem_type)


Training samples: 2734420, Testing samples: 143917
Number of labels: 174


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


训练集: 50000 样本, 测试集: 100 样本
Model initialized: DistilBertForSequenceClassification
Num labels: 174
Problem type: multi_label_classification
Model initialized: DistilBertForSequenceClassification
Num labels: 174
Problem type: multi_label_classification


In [11]:
# 实现 checkpoint 保存与加载工具函数

from typing import Dict, Any


def save_checkpoint(state: Dict[str, Any], filename: Path):
    """保存训练状态到指定文件。

    state 示例:
    {
        'epoch': int,
        'global_step': int,
        'model_state': model.state_dict(),
        'optimizer_state': optimizer.state_dict(),
        'best_metric': float
    }
    """
    filename = Path(filename)
    filename.parent.mkdir(parents=True, exist_ok=True)
    torch.save(state, filename)
    print(f"Checkpoint saved to {filename}")


def load_checkpoint(model, optimizer, filename: Path):
    """从指定文件加载训练状态，返回 (start_epoch, global_step, best_metric)。"""
    filename = Path(filename)
    if not filename.exists():
        print(f"No checkpoint found at {filename}, start from scratch.")
        return 0, 0, None

    ckpt = torch.load(filename, map_location=device)
    model.load_state_dict(ckpt["model_state"])
    if optimizer is not None and "optimizer_state" in ckpt:
        optimizer.load_state_dict(ckpt["optimizer_state"])
    start_epoch = ckpt.get("epoch", 0)
    global_step = ckpt.get("global_step", 0)
    best_metric = ckpt.get("best_metric", None)
    print(f"Loaded checkpoint from {filename}: epoch={start_epoch}, global_step={global_step}, best_metric={best_metric}")
    return start_epoch, global_step, best_metric


# 简单测试：仅在第一次运行时保存一个空的 state 示例
if not LAST_CKPT.exists():
    dummy_state = {
        "epoch": 0,
        "global_step": 0,
        "model_state": model.state_dict(),
        "best_metric": None,
    }
    save_checkpoint(dummy_state, LAST_CKPT)
else:
    print("Existing checkpoint found, skip dummy save.")


Existing checkpoint found, skip dummy save.


In [12]:
# 训练设置与 Trainer 初始化（集成 checkpoint 保存与自动加载）

from transformers import TrainingArguments, Trainer
from sklearn.metrics import f1_score

# 指标阈值
def sigmoid_np(x):
    return 1 / (1 + np.exp(-x))

THRESH = 0.4  # 多标签预测阈值


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = sigmoid_np(logits)
    preds = (probs > THRESH).astype(int)

    exact_match = (preds == labels).all(axis=1).mean()
    f1_micro = f1_score(labels, preds, average="micro", zero_division=0)
    f1_macro = f1_score(labels, preds, average="macro", zero_division=0)

    return {
        "exact_match": exact_match,
        "f1_micro": f1_micro,
        "f1_macro": f1_macro,
    }


output_dir = CHECKPOINT_DIR  # 直接复用 checkpoint 目录

training_args = TrainingArguments(
    output_dir=str(output_dir),
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    learning_rate=3e-5,
    eval_strategy="epoch",
    save_strategy="no",  # 由我们自定义 save_checkpoint，不使用 HF 内建保存
    logging_steps=20,
    load_best_model_at_end=False,
    report_to="none",
    use_mps_device=(device.type == "mps"),
    fp16=False,
    dataloader_num_workers=0,
)

optimizer = torch.optim.AdamW(model.parameters(), lr=training_args.learning_rate)

# 自动加载最新 checkpoint（如果存在）
start_epoch, global_step, best_metric = load_checkpoint(model, optimizer, LAST_CKPT)
if best_metric is None:
    best_metric = 0.0

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("Trainer initialized. start_epoch=", start_epoch, "global_step=", global_step, "best_metric=", best_metric)


/Users/jylu/Projects/acafeed/.venv/lib/python3.14/site-packages/transformers/training_args.py:2301: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(


Loaded checkpoint from model/checkpoint_model1/model1_last.pt: epoch=0, global_step=0, best_metric=None
Trainer initialized. start_epoch= 0 global_step= 0 best_metric= 0.0


/var/folders/44/62vjt8dx0sd5w6cnwjv2k5gc0000gn/T/ipykernel_19304/816535516.py:54: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
# 主训练循环：支持重复运行时自动从 checkpoint 恢复

num_epochs = training_args.num_train_epochs

print(f"Start training from epoch {start_epoch} / {num_epochs}")

best_metric_local = best_metric

for epoch in range(int(start_epoch), int(num_epochs)):
    print("=" * 40)
    print(f"Epoch {epoch+1}/{num_epochs}")

    # Trainer 自带训练循环，但我们希望在 epoch 粒度上保存 ckpt
    train_result = trainer.train()
    global_step = train_result.global_step

    eval_metrics = trainer.evaluate()
    print("Eval metrics:", eval_metrics)

    # 保存最新 checkpoint
    state = {
        "epoch": epoch + 1,
        "global_step": int(global_step),
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "best_metric": float(best_metric_local),
    }
    save_checkpoint(state, LAST_CKPT)

    # 根据 f1_micro 更新 best checkpoint
    current_metric = eval_metrics.get("f1_micro", 0.0)
    if current_metric > best_metric_local:
        best_metric_local = current_metric
        state["best_metric"] = float(best_metric_local)
        save_checkpoint(state, BEST_CKPT)
        print(f"New best f1_micro={best_metric_local:.4f}, saved to BEST_CKPT")

print("Training finished. Best f1_micro=", best_metric_local)


Start training from epoch 0 / 3
Epoch 1/3


/Users/jylu/Projects/acafeed/.venv/lib/python3.14/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# 推理与多标签预测示例（使用 best 或 last checkpoint）

# 优先加载最佳 checkpoint
if BEST_CKPT.exists():
    load_checkpoint(model, optimizer=None, filename=BEST_CKPT)
else:
    load_checkpoint(model, optimizer=None, filename=LAST_CKPT)

model.to(device)
model.eval()

sample_titles = [
    filtered_df.iloc[0]["Title"],
    filtered_df.iloc[10]["Title"],
    filtered_df.iloc[100]["Title"],
]

encoded = tokenizer(sample_titles, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
if "token_type_ids" in encoded:
    encoded.pop("token_type_ids")

encoded = {k: v.to(device) for k, v in encoded.items()}

with torch.no_grad():
    outputs = model(**encoded)
    probs = torch.sigmoid(outputs.logits).cpu()

for title, prob in zip(sample_titles, probs):
    pred_indices = (prob > THRESH).nonzero(as_tuple=True)[0].tolist()
    pred_labels = [(id2label[i], float(prob[i])) for i in pred_indices]
    top5 = sorted([(id2label[i], float(p)) for i, p in enumerate(prob)], key=lambda x: x[1], reverse=True)[:5]

    print("\n" + "=" * 80)
    print("Title:", title[:100])
    print(f"预测标签 (>{THRESH}): {len(pred_labels)} 个")
    for label, score in pred_labels:
        print(f"  - {label}: {score:.3f}")
    print("\nTop 5 标签:")
    for label, score in top5:
        print(f"  - {label}: {score:.3f}")
